# Capitation Cross Validation
Security Note:  No PII, PHI, or Company identifiable information, and data structures have been masked. 
 
### Purpose: 
A quicker solution to cross-validate capitation payments and adjustments between tables. Previously, these tables were not linked because our production data flowed into them at different times and each table stored and outputted the data slightly. Described in a non-technical way, imagine if you were shopping for apples, but all the labels indicating which type of apple and the price tag had been separately randomized. This solved a painful problem for my associate director. It also built traceability to the general ledger and readiness for potential database migrations.  
 
### Key Concepts
* Parameters
* Temp Table operations
* Inner Joins
* Case statements
* Sub-Queries 
* Update statements
* Window Functions 
* Unions
 
### Walkthrough:
1. Identify goal/mock up of expected results
2. Data profiling; identify linkable, filterable, and final returned fields, this is also where I drafted how I could cast, parse, or use some other type of data transformation 
3. Subset Data into temp tables and/or subqueries, this is where I determined how tables would differentiate between main payment and adjustment values
4. Select desired data from joined tables
5. Test data


In [ ]:
-- query Table_1 
*/
USE [Prod_Table]
IF object_id('tempdb..##tCap') IS NOT NULL 
DROP TABLE ##tCap;

declare @Proc_Start_INT int = 20240801
declare @Proc_Start date = '2024-08-01'
declare @Proc_End date = '2024-08-31'

SELECT Client, FileDate, TransType, SubType, SUM(CapAmount) AS CapAmount, TTL
INTO ##tCap
FROM 
(
	SELECT Client, FileDate, 'Cap Payment' AS TransType, 'N/A' AS SubType, sum(GroupPayment) AS CapAmount, CAST(null as numeric(15,2)) AS TTL
	FROM [dbo].[Table_1]
	JOIN [dbo].[Contract_Table]  on Client = [Contract_Table].ConsolidatedID and @Proc_Start between [Contract_Table].policyStart and [Contract_Table].PolicyEnd 
	WHERE 
		FileDate between @Proc_Start and @Proc_End
		and GroupPayment <> 0 
		and PaymentAmount <> 0 
		and Client <> 'ClientID'
		and PS_CATEGORY_ID <> ('PAYABLE')
	GROUP BY Client, FileDate 

	UNION

	SELECT Client, FileDate, 'Cap Adjustment' AS TransType, 
	CASE WHEN Client = 'ClientID' AND LEFT(ADJ_BNEFT_CODE,2) = 'VP' THEN 'ADJ_BNEFT_CODE:VP1-VP5' ELSE 'Allocated Provider Adjustments' END AS SubType,
	sum(AllocatedAdjustment) AS CapAmount, CAST(null as numeric(15,2)) AS TTL
	FROM [dbo].[Table_2]
	JOIN [dbo].[Contract_Table]  on Client = [Contract_Table].ConsolidatedID and @Proc_Start between [Contract_Table].policyStart and [Contract_Table].PolicyEnd 
	WHERE 
		FileDate between @Proc_Start and @Proc_End
		--and GroupPayment <> 0 
		and Client <> 'ClientID'
		and PS_CATEGORY_ID IN('PAID', 'PAYABLE')
	GROUP BY Client, FileDate, 
	CASE WHEN Client = 'ClientID' AND LEFT(ADJ_BNEFT_CODE,2) = 'VP' THEN 'ADJ_BNEFT_CODE:VP1-VP5' ELSE 'Allocated Provider Adjustments' END 
) AS IntermediateCap
GROUP BY Client, FileDate, TransType, SubType, TTL
;

/*
create a sub query with the aggregated values and the keys to link back to the table to be updated.  You then link the tables together in the update statement.
*/

update aa
set aa.[TTL] = bb.[TTL] 
from ##tCap AS aa
Join
	(
	SELECT Client, FileDate,  SUM(CapAmount) AS TTL
	FROM ##tCap
	WHERE SubType != 'ADJ_BNEFT_CODE:VP1-VP5'
	GROUP BY 
	Client,
	FileDate	
	) AS bb

ON aa.Client = bb.Client
AND aa.FileDate = bb.FileDate
where aa.SubType != 'ADJ_BNEFT_CODE:VP1-VP5' 
AND aa.TTL IS NULL;

update ##tCap
set [TTL] = CapAmount
from ##tCap
where SubType = 'ADJ_BNEFT_CODE:VP1-VP5' 

--SELECT * FROM ##tcap;
/*
-------------------------------------------------
query Table_3
*/

IF object_id('tempdb..##Table_3') IS NOT NULL 
DROP TABLE ##Table_3;

--declare @Proc_Start_INT int = 20231201
--declare @Proc_Start date = '2023-12-01'
--declare @Proc_End date = '2023-12-31'

SELECT Client, FileDate, FileNo, SubType, Table_3Amt, Table_3TTL
INTO ##Table_3
FROM 
(
	SELECT Client, FileDate, FileNo, SUM(Amount) AS Table_3Amt, 'N/A' AS SubType, CAST(null as numeric(15,2)) AS Table_3TTL
	FROM [dbo].[Table_3]
	WHERE 
		filetype = 'Cap' 
		and recordstamp between @Proc_Start and @Proc_End
		and member IS NOT NULL
	GROUP BY Client, FileDate, FileNo
	
	UNION

	SELECT Client, FileDate, FileNo, SUM(Amount) AS Table_3Amt,
	CASE WHEN Client = 'ClientID' AND GLCategory = 'ADJ_BNEFT_CODE:VP1-VP5' THEN 'ADJ_BNEFT_CODE:VP1-VP5' ELSE 'Allocated Provider Adjustments' END AS SubType, 
	CAST(null as numeric(15,2)) AS Table_3TTL
	FROM [dbo].[Table_3]
	WHERE 
		filetype = 'Cap' 
		and recordstamp between @Proc_Start and @Proc_End
		and member IS NULL
	GROUP BY Client, FileDate, FileNo, 
	CASE WHEN Client = 'ClientID' AND GLCategory = 'ADJ_BNEFT_CODE:VP1-VP5' THEN 'ADJ_BNEFT_CODE:VP1-VP5' ELSE 'Allocated Provider Adjustments' END
	
) AS Intermediate[Banking_Table]Detail

update cc
set cc.[Table_3TTL] = dd.[Table_3TTL] 
from ##Table_3 AS cc
Join
	(
	SELECT Client, FileDate, FileNo, SUM(Table_3Amt) AS Table_3TTL
	FROM ##Table_3
	GROUP BY 
	Client,
	FileDate,
	FileNo
	) AS dd

ON cc.Client = dd.Client
AND cc.FileDate = dd.FileDate
AND cc.Table_3TTL IS NULL;
	
/*
-------------------------------------------------
query Table_4 
*/

IF object_id('tempdb..##Table_4') IS NOT NULL 
DROP TABLE ##Table_4;

--declare @Proc_Start_INT int = 20231201
--declare @Proc_Start date = '2023-12-01'
--declare @Proc_End date = '2023-12-31'

SELECT Client, Table_4Total, FileType, SubType
INTO ##Table_4
FROM 
(
	
	SELECT Client, SUM(Amount) AS Table_4Total, FileType,
	CASE WHEN Client = 'ClientID' AND ClientName = 'Client_Name - BenefitCode' THEN 'ADJ_BNEFT_CODE' ELSE 'N/A' END AS SubType
	FROM [dbo].[Table_4]
	WHERE 
		filetype = 'Cap' 
		and recordstamp between @Proc_Start and @Proc_End
	GROUP BY Client, FileDate, FileType,
	CASE WHEN Client = 'ClientID' AND ClientName = 'Client_Name - BenefitCode' THEN 'ADJ_BNEFT_CODE' ELSE 'N/A' END
	HAVING CASE WHEN Client = 'ClientID' AND ClientName = 'Client_Name - BenefitCode' THEN 'ADJ_BNEFT_CODE' ELSE 'N/A' END = 'N/A'

	UNION

	SELECT Client, SUM(Amount) AS Table_4Total, FileType,
	CASE WHEN Client = 'ClientID' AND ClientName = 'Client_Name - BenefitCode' THEN 'ADJ_BNEFT_CODE' ELSE 'Allocated Provider Adjustments' END AS SubType
	FROM [dbo].[Table_4]
	WHERE 
		filetype = 'Cap' 
		and recordstamp between @Proc_Start and @Proc_End
	GROUP BY Client, FileDate, FileType,
	CASE WHEN Client = 'ClientID' AND ClientName = 'Client_Name - BenefitCode' THEN 'ADJ_BNEFT_CODE' ELSE 'Allocated Provider Adjustments' END

) AS Intermediate[Banking_Table]Approval
;

/*
-------------------------------------------------
query Table_5
*/

IF object_id('tempdb..##Table_5') IS NOT NULL 
DROP TABLE ##Table_5;

--declare @Proc_Start_INT int = 20231201
--declare @Proc_Start date = '2023-12-01'
--declare @Proc_End date = '2023-12-31'

SELECT PolicyNo AS Client, CheckNo, SUM(Amount) AS CheckAmt, Series, TransType AS TransTypeCode
INTO ##Table_5
FROM [dbo].[Table_5]
WHERE 
	Series = 'NN' 
	and TransType = '0300'
	and recordstamp between @Proc_Start and @Proc_End
GROUP BY PolicyNo, CheckNo, Series, TransType
;

/*
-------------------------------------------------
query Table_6
*/

--declare @Proc_Start_INT int = 20231201
--declare @Proc_Start date = '2023-12-01'
--declare @Proc_End date = '2023-12-31'

IF object_id('tempdb..##Table_6') IS NOT NULL 
DROP TABLE ##Table_6;

SELECT SUM(Amount) AS Table_6Amt, CAST(Uploaded AS Date) AS Uploaded, RecCount, FileNo /*FilNo, CreateDate, RecordStamp are potential foreign keys)*/
INTO ##Table_6
FROM [dbo].[Table_6]
WHERE 
recordstamp between @Proc_Start and @Proc_End
GROUP BY CAST(Uploaded AS Date), RecCount, FileNo
;

-------------------------------------------------

SELECT 
##tCap.Client,
##tCap.FileDate,
##tCap.TransType,
##tCap.SubType,
FORMAT(##TCap.CapAmount, 'C') AS CapAmount,
FORMAT(##tCap.TTL, 'C') AS TTL,
##Table_3.FileNo,
FORMAT(##Table_3.Table_3Amt, 'C') AS Table_3Amt,
FORMAT(##Table_3.Table_3TTL, 'C') AS Table_3TTL,
FORMAT(##Table_4.Table_4Total, 'C') AS Table_4Total, 
##Table_4.FileType,
##Table_5.CheckNo,
FORMAT(##Table_5.CheckAmt, 'C') AS CheckAmt,
##Table_5.Series,
##Table_5.TransTypeCode,
FORMAT(##Table_6.Table_6Amt, 'C') AS Table_6Amt,
##Table_6.Uploaded, 
##Table_6.RecCount

FROM ##tCap 
JOIN ##Table_3 ON ##tCap.Client = ##Table_3.Client AND ##tCap.SubType = ##Table_3.SubType
JOIN ##Table_4 ON ##tCap.Client = ##Table_4.Client AND ##tCap.SubType = ##Table_4.SubType
JOIN ##Table_5 ON ##tCap.Client = ##Table_5.Client
JOIN ##Table_6 ON ##Table_3.FileNo = ##Table_6.FileNo
ORDER BY ##tCap.Client, ##tCap.FileDate, ##tCap.TransType, ##tCap.SubType
;

Why global temp tables?
* global temp can be queried across instances and is visible to other users 
* temp table uses 
    * querying a subset of data may be faster than original dataset
    * stored on server in tempdb (production server is shared with limited space)
    * declutter object explorer space
    * save physical storage
	